In [ ]:
!pip install datasets transformers rouge_score
!pip install pytube pysrt moviepy SpeechRecognition pydub
!pip install torchmetrics
!pip install contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 31.7 MB/s 
     |████████████████████████████████| 4.7 MB 64.0 MB/s 
     |████████████████████████████████| 115 kB 67.1 MB/s 
     |████████████████████████████████| 101 kB 12.1 MB/s 
     |████████████████████████████████| 212 kB 73.8 MB/s 
     |████████████████████████████████| 127 kB 55.7 MB/s 
     |████████████████████████████████| 6.6 MB 63.7 MB/s 
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=e1617265acd053529eeae885614c921b2f1e189a12ce6b33b01cd3ee2e82f775
  Stored in directory: /root/.cache/pip/wheels/84/ac/6b/38096e3c5bf1dc87911e3585875e21a3ac610348e740409c76
Successfully built rouge-score
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/s

In [ ]:
import os
import csv
import pandas as pd
import matplotlib.pyplot as plt
from transformers import pipeline, TFAutoModelWithLMHead, AutoTokenizer
from datasets import load_dataset, load_metric
import contractions

In [ ]:
model = TFAutoModelWithLMHead.from_pretrained("sshleifer/distilbart-xsum-1-1")
tokenizer = AutoTokenizer.from_pretrained("sshleifer/distilbart-xsum-1-1")

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_tf_auto.py:573: FutureWarning: The class `TFAutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `TFAutoModelForCausalLM` for causal language models, `TFAutoModelForMaskedLM` for masked language models and `TFAutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


All model checkpoint layers were used when initializing TFBartForConditionalGeneration.

All the layers of TFBartForConditionalGeneration were initialized from the model checkpoint at sshleifer/distilbart-xsum-1-1.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartForConditionalGeneration for predictions without further training.


In [ ]:
rouge = load_metric('rouge')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
transcript='/content/drive/MyDrive/How2 Dataset/text/sum_train/tran.tok.txt'
summary='/content/drive/MyDrive/How2 Dataset/text/sum_train/desc.tok.txt'

In [ ]:
import re
transcripts=[]
summaries=[]

In [ ]:
with open(summary, "r") as file1:
    FileContent = file1.read()

    s=re.split(r"\n", FileContent)

for i in range(50):
  rs=""
  s[i] = re.sub(r'.', '', s[i], count = 11)
  expanded_words=[]
  for word in s[i].split():
      expanded_words.append(contractions.fix(word))
  expanded_text = ' '.join(expanded_words)
  words = re.findall(r'\w+', expanded_text.lower())
  rs= ' '.join(words)
  summaries.append(rs)

summaries[0:10]

['when choosing a martial art choose an informed and talented instructor choose a martial arts fighting system with tips from a martial arts teacher in this free self defense video',
 'using front layers as a guide cut back layers into curly hair learn to cut layers in curly hair with expert tips from a hairstylist in this free video about curly hairstyles',
 'the intonation on a guitar means a lot in its playability learn about all the facets that you can modify to adjust your guitar s action in this free video clip',
 'leveling out tracks is another way of evening out the sound learn how to level out beats in ultrabeat using logic 8 in this free music software video from a music studio owner and producer',
 'learn how to use a saw to cut gourds for decorative gourd art in this free online video arts and crafts lesson',
 'sparkling wines of spain produced in catalonia are called cava learn about spanish sparkling wines with tips from a sommelier in this free sparkling wine video',
 'm

In [ ]:
with open(transcript, "r") as file1:
    FileContent = file1.read()

    s=re.split(r"\n", FileContent)
for i in range(50):

  rs=""
  s[i] = re.sub(r'.', '', s[i], count = 11)
  expanded_words=[]
  for word in s[i].split():
      expanded_words.append(contractions.fix(word))
  expanded_text = ' '.join(expanded_words)
  words = re.findall(r'\w+', expanded_text.lower())
  rs= ' '.join(words)

  transcripts.append(rs)

transcripts[0:10]


['hi everybody i m john graden from the martial arts teacher s association and johngraden com one of the questions i have been asked through the years how do you choose a martial art it really varies it depends on what your goal is are you trying to get in better shape do you want to learn self defense do you want to compete or are you just interested in the mental benefits of the martial arts offer you mental benefits such as self confidence self discipline self control respect so there is a lot of variables in my mind when people say what style is best what style should i do the only style that is really important is the teaching style of the instructor so how do you choose a good martial arts it is really more about choosing a good martial arts instructor it s about choosing a good martial arts school so when you are going to make the decision make sure first you know what your goal is if you really want a strong self defense oriented system you want to be working with an instructor

In [ ]:
articles=transcripts
summary=summaries

In [ ]:
import re
def getLength(art):
  res = len(re.findall(r'\w+', art))
  return res

In [ ]:
len_art=[]
len_sum=[]
for i in articles:
  len_art.append(getLength(i))
for i in summary:
  len_sum.append(getLength(i))

In [ ]:
max(len_art)

835

In [ ]:
max(len_sum)

62

In [ ]:
avg_art=sum(len_art)/len(len_art)
avg_sum=sum(len_sum)/len(len_sum)

In [ ]:
print(avg_art,avg_sum)

272.0 33.74


In [ ]:
def summarize(model, tokenizer, input_text, actual_summary):
    inputs = tokenizer.encode("summarize: " + input_text, return_tensors="tf", max_length=max(len_art), truncation=True)
    outputs = model.generate(inputs, max_length=max(len_sum), min_length=round(avg_sum), length_penalty=2.0, num_beams=4, early_stopping=True,  no_repeat_ngram_size=3)
    pred_summary = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in outputs]
    return pred_summary[0]

In [ ]:
pred_summary = []
for i in range(50):
    pred_summary.append(summarize(model, tokenizer, articles[i], summary[i]))

In [ ]:
def computeScore(score):
  precision = []
  recall = []
  f1 = []
  for i in range(50):
      result = rouge.compute(predictions=[pred_summary[i]], references=[summary[i]])
      precision.append(result[score].high.precision)
      recall.append(result[score].high.recall)
      f1.append(result[score].high.fmeasure)
  pavg = sum(precision)/len(precision)
  print("_______",score,"____________")
  print('Precision:', pavg)

  ravg = sum(recall)/len(recall)
  print('Recall', ravg)

  favg = sum(f1)/len(f1)
  print('F1- Score', favg)
  # plotGraph(precision,recall,f1)
  print("______________________________")

In [ ]:
# indices=[i for i in range(50)]
# def plotGraph(precision,recall,f1):
#   plt.plot(indices, precision, label = "precision")
#   plt.plot(indices, recall, label = "recall")
#   plt.plot(indices, f1, label = "f1-score")
#   plt.xlabel('id')
#   plt.ylabel('score')
#   plt.title('ROUGE Score')

In [ ]:
computeScore('rouge1') #with normal

_______ rouge1 ____________
Precision: 0.14303041975529493
Recall 0.11254187342021708
F1- Score 0.12026603100517956
______________________________


In [ ]:
from torchmetrics.functional.text.bert import bert_score
preds = summary
target = pred_summary
score = bert_score(preds, target)
from pprint import pprint
rounded_score = {k: [round(v, 3) for v in vv] for k, vv in score.items()}
pprint(rounded_score)

/usr/local/lib/python3.7/dist-packages/torchmetrics/functional/text/bert.py:547: UserWarning: The argument `model_name_or_path` was not specified while it is required when default `transformers` model are used.It is, therefore, used the default recommended model - roberta-large.
  "The argument `model_name_or_path` was not specified while it is required when default"


Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessiv

{'f1': [0.967,
        0.964,
        0.959,
        0.956,
        0.967,
        0.957,
        0.965,
        0.968,
        0.962,
        0.966,
        0.947,
        0.962,
        0.967,
        0.965,
        0.963,
        0.956,
        0.959,
        0.961,
        0.969,
        0.96,
        0.954,
        0.955,
        0.962,
        0.961,
        0.958,
        0.957,
        0.961,
        0.97,
        0.956,
        0.964,
        0.959,
        0.96,
        0.953,
        0.958,
        0.957,
        0.957,
        0.961,
        0.944,
        0.942,
        0.963,
        0.953,
        0.947,
        0.938,
        0.959,
        0.947,
        0.958,
        0.955,
        0.956,
        0.944,
        0.965],
 'precision': [0.97,
               0.966,
               0.965,
               0.946,
               0.964,
               0.959,
               0.956,
               0.968,
               0.958,
               0.964,
               0.955,
           

In [ ]:
sum(rounded_score['f1'])/50

0.9582800000000004

In [ ]:
sum(rounded_score['precision'])/50

0.9550200000000002

In [ ]:
sum(rounded_score['recall'])/50

0.9617800000000003